In [33]:
import pandas as pd
import json

In [50]:
json_index = []
json_ids = []

for diz in ['00','01','02','03','04','05','06','07','08','09','10']:
    for unit in ['0','1','2','3','4','5','6','7','8','9']:
        if diz+unit != "000":
            ids = pd.read_json("data/wiki-pages/wiki-"+diz+unit+".jsonl", lines=True)['id'].tolist()
            json_ids.extend(ids)
            for i in range(len(ids)):
                json_index.append(diz+unit)

data = pd.DataFrame(json_index,columns=["json_index"])
data.insert(1,"json_ids",json_ids)

data

,json_index,json_ids
0,001,
1,001,1928_in_association_football
2,001,1986_NBA_Finals
3,001,1901_Villanova_Wildcats_football_team
4,001,1992_Northwestern_Wildcats_football_team
...,...,...
5416532,109,Yuto_Agarie
5416533,109,Yume_1_Go
5416534,109,Yada_Yada_-LRB-album-RRB-
5416535,109,Xylorycta_bipunctella


In [57]:
data.to_csv("data/wiki_indexes.csv",index=False)

In [82]:
data2 = pd.read_json("data/train.jsonl", lines=True)
wiki_page = data2["evidence"][1000][0][0][2]

json_index = data['json_index'][data['json_ids']==wiki_page].to_string().split(" ").pop()

wiki_data = pd.read_json("data/wiki-pages/wiki-"+json_index+".jsonl", lines=True)

wiki_article_lines = wiki_data["lines"][wiki_data["id"]==wiki_page].to_list()[0].split("\n")

print(wiki_article_lines[])

['0\tTransformers : Age of Extinction is a 2014 American science fiction action film based on the Transformers franchise .\tTransformers\tTransformers\tscience fiction\tScience fiction film\taction film\taction film', "1\tIt is the fourth installment of the live-action Transformers film series and a stand-alone sequel to 2011 's Transformers : Dark of the Moon , taking place five years after its events .\tTransformers\tTransformers\tlive-action\tLive action", '2\tLike its predecessors , it was directed by Michael Bay and written by Ehren Kruger , with Steven Spielberg and Bay as executive producers .\tMichael Bay\tMichael Bay\tEhren Kruger\tEhren Kruger\tSteven Spielberg\tSteven Spielberg', '3\tIt stars Mark Wahlberg , with Peter Cullen reprising his role as the voice of Optimus Prime , as the lead roles .\tMark Wahlberg\tMark Wahlberg\tPeter Cullen\tPeter Cullen\tOptimus Prime\tOptimus Prime', '4\tIt is the first film in the series to not feature the original human characters from the

In [12]:
import torch
import json
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

model_token_verdict = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model_voter_verdict = DistilBertForSequenceClassification.from_pretrained("model/voter_5")

votes = [[2, 0, 0, 3, 0], [2, 2, 2, 3, 2], [2, 0, 1, 3, 1], [2, 0, 2, 3, 1], [1, 2, 2, 3, 2]]


inputs = model_token_verdict(json.dumps(votes), return_tensors="pt")
with torch.no_grad():
    logits = model_voter_verdict(**inputs).logits

predicted_class_id = logits.argmax().item()

classes = ['NOT_ENOUGH_INFO','SUPPORTS', 'REFUTES', None]

print(classes[predicted_class_id])

DISPUTED
